In [17]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# ⚙️ Setup navegador
def setup_driver():
    options = Options()
    # options.add_argument("--headless")  # Activa si no necesitas ver el navegador
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 🍪 Aceptar cookies
def aceptar_cookies(driver):
    try:
        wait = WebDriverWait(driver, 10)
        boton = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button#didomi-notice-agree-button")))
        boton.click()
        print("🍪 Cookies aceptadas.")
        time.sleep(2)
    except:
        print("⚠️ Cookies ya aceptadas o no detectadas.")

# 🧩 Extraer detalles de la página
def extraer_detalles(driver):
    detalles = {}

    try:
        conservacion = driver.find_element(By.XPATH, "//span[contains(., 'Conservación')]/following-sibling::span").text.strip()
        detalles["conservacion"] = conservacion
    except:
        detalles["conservacion"] = None

    try:
        precio_m2 = driver.find_element(By.CSS_SELECTOR, "li.features-summary__item:nth-child(5)").text.strip()
        detalles["precio_m2"] = precio_m2
    except:
        detalles["precio_m2"] = None


    return detalles

# 📥 Leer URLs desde archivo de venta
df = pd.read_csv(r"C:\Users\pablo\proyecto\comparador\venta_madrid.csv")
urls = df["link"].dropna().unique().tolist()

# 🚀 Iniciar scraping
driver = setup_driver()
resultados = []

print(f"🔍 Scrapeando detalles para {len(urls)} inmuebles de venta...\n")

for i, url in enumerate(urls, 1):
    print(f"{i}/{len(urls)} → {url}")
    driver.get(url)
    time.sleep(2)  # deja que cargue el banner de cookies

    if i == 1:
        aceptar_cookies(driver)

    detalles = extraer_detalles(driver)
    detalles["url"] = url
    resultados.append(detalles)
    time.sleep(1)

driver.quit()

# 💾 Guardar resultados
df_detalles = pd.DataFrame(resultados)
df_detalles.to_csv("venta_detalles_scrapeados.csv", index=False)
print("\n✅ Detalles guardados en venta_detalles_scrapeados.csv")



🔍 Scrapeando detalles para 2005 inmuebles de venta...

1/2005 → https://www.pisos.com/comprar/casa_adosada-estacion_zona_norte28224-5106780590_109700/
🍪 Cookies aceptadas.
2/2005 → https://www.pisos.com/comprar/piso-san_sebastian_de_los_reyes_casco_antiguo28703-53387680450_515177/
3/2005 → https://www.pisos.com/comprar/piso-zona_sureste-5129162041_109700/
4/2005 → https://www.pisos.com/comprar/piso-hoyo_de_manzanares_centro_urbano-46711883004_994129/
5/2005 → https://www.pisos.com/comprar/piso-sol_barrio28013-53433152926_524671/
6/2005 → https://www.pisos.com/comprar/piso-arroyos_y_tempranales28702-41742370937_106000/
7/2005 → https://www.pisos.com/comprar/piso-pinar_punta_galea28290-53410897274_100500/
8/2005 → https://www.pisos.com/comprar/piso-guindalera28028-51765010516_440000/
9/2005 → https://www.pisos.com/comprar/piso-fuente_del_berro28028-631974332_440000/
10/2005 → https://www.pisos.com/comprar/chalet-aldea_del_fresno_centro_urbano-43379000866_101800/
11/2005 → https://www.pis